In [27]:


import glob 
svpath = "F:/PythonProjects/NietoExcercise-1/Works/WorksSeeds/"
resultStrings = glob.glob(svpath+"savedBestSeed-18*")
#resultStrings = glob.glob("F:/PythonProjects/NietoExcercise-1/SavedResults/TestFold/savedBestSeed-5*")
#resultStrings = glob.glob("F:/PythonProjects/NietoExcercise-1/SavedResultsNN/sa*")
#skipsubj = True
skipSubjects = []
onlySignificant =             True
minAcc = 0.55



In [28]:
#!pip3 install pandas -q

import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in resultStrings:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)],           dtype=object)

    currSeed = subject[1]
    currSubj = subject[2]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[3]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC])
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][index][1] * (
                        1-(1/nrOfSeedsPerSubject[currSubj])) + ares*(1/nrOfSeedsPerSubject[currSubj])

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aLz, areg, adrp] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp])
                    nrOfCombination += 1
                #combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp])
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][index][1] * (
                        1-(1/nrOfSeedsPerSubject[currSubj])) + ares*(1/nrOfSeedsPerSubject[currSubj])

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

#headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()
                    # delete(topTen[0])

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage/nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option('display.max_columns', None)
table = pd.DataFrame(np.array(subjectResultArray), columns=headers)
display(table)


(1, 77)
(10, 77)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3160\1791118945.py:162: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage/nrOfSubjects


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.75,0.7,0.7,0.7,0.75,0.75,0.7,0.6,None,None,None,0.8,0.65,0.75,0.8,0.75,0.75,0.55,0.8,0.75,0.65,0.6,0.6,None,None,None,None,None,None,0.85,0.85,0.9,0.9,0.75,0.65,0.75,0.75,0.8,0.8,0.75,0.75,0.6,0.6,0.6,0.75,0.7,0.7,0.7,0.75,0.75,0.7,0.65,0.75,0.65,None,0.75,0.65,0.55,0.8,0.8,0.75,None,0.75,0.8,0.8,0.75,0.75,0.65,0.7,0.75,0.75,0.8,0.55,0.8,0.75,0.55,0.55
2,None,None,None,None,None,None,None,0.666667,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,None,0.625,0.708333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.555556,None,None,0.611111,None,None,None,None,None,None,None,None,None,None,0.555556,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,0.625,0.666667,None,None,None,None,None,None,None,None,None,None,0.625,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,0.625,0.75,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,0.583333,0.583333,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None
6,0.590909,None,0.590909,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.590909,0.590909,0.636364,None,None,None,None,None,None,0.590909,None,None,0.590909,None,None,None,None,None,None,0.590909,None,0.636364,None,None,None,None,None,None,None,None,None,None,0.590909,None,None,None,None,None,None,None,None,None,None,0.590909,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,0.583333,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,0.55,None,None,None,None,None,None,None,0.65,None,None,None,0.6,None,0.6,0.6,0.65,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.55,0.6,0.6,None,None,None,0.55,None,None,None,None,None,0.6,None,None,None,None,None,None,None,None,None,None,0.6,None,None,None,None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,Non

In [29]:
print("Top ten highest accuracies")
topTen.reverse()
for top in topTen:
    print(f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}")

Top ten highest accuracies
Accuracy 0.9 : subject 1, feature:dataWCV, kernelsigmoid, C:5.0
Accuracy 0.9 : subject 1, feature:dataWCV, kernelsigmoid, C:3.88
Accuracy 0.85 : subject 1, feature:dataWCV, kernelsigmoid, C:2.75
Accuracy 0.85 : subject 1, feature:dataWCV, kernelsigmoid, C:1.62
Accuracy 0.8 : subject 1, feature:dataHRCVdataCV, kernelrbf, C:5.0
Accuracy 0.8 : subject 1, feature:dataHRCVdataCV, kernelrbf, C:3.88
Accuracy 0.8 : subject 1, feature:dataCVdataWCV, kernelrbf, C:2.75
Accuracy 0.8 : subject 1, feature:dataCV, kernelsigmoid, C:1.62
Accuracy 0.8 : subject 1, feature:dataCV, kernelrbf, C:2.75
Accuracy 0.8 : subject 1, feature:dataCV, kernellinear, C:0.5


In [30]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option('display.max_columns', None)
table = pd.DataFrame(np.array(comboAverage) ,columns = headers)
display(table)


print(f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}")
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(maxInd)
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")

[0.         0.71969697 0.625      0.57407407 0.625      0.58333333
 0.59917355 0.58333333 0.59583333 0.58333333]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76
0,0.670455,0.7,0.645455,0.7,0.6875,0.708333,0.7,0.633333,0.583333,NaN,0.55,0.8,0.65,0.666667,0.8,0.75,0.6875,0.55,0.725,0.62963,0.65,0.6,0.598611,NaN,0.6,0.591414,0.620455,0.609848,NaN,0.85,0.85,0.9,0.741667,0.62963,0.620455,0.75,0.6875,0.67197,0.775,0.75,0.75,0.575,0.6,0.6,0.670455,0.7,0.639899,0.625,0.75,0.75,0.7,0.619444,0.75,0.625,0.708333,0.75,0.65,0.570455,0.8,0.8,0.75,NaN,0.75,0.8,0.7,0.666667,0.666667,0.65,0.645455,0.75,0.666667,0.655556,0.55,0.8,0.75,0.55,0.5875


Max average accuracy 0.8999999999999999
31
dataWCV
sigmoid
C: 3.88


In [31]:
table = pd.DataFrame(np.array(comboTotal) ,columns = headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal)==False)]))
maxInd2 = int(np.where(comboTotal == maxVal2)[1])
print(maxInd2)
print(nrOfSubjects[maxInd2])
print(f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} ")

print(list(nameDict.keys())[combinationList[maxInd2][0]])
print(list(kernelDict.keys())[combinationList[maxInd2][1]])
print(f"C: {combinationList[maxInd2][2]} ")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76
0,1.340909,0.7,1.290909,0.7,1.375,1.416667,0.7,1.266667,0.583333,0.0,0.55,0.8,0.65,1.333333,0.8,0.75,1.375,0.55,1.45,1.888889,0.65,0.6,2.394444,0.0,0.6,1.774242,1.240909,1.219697,0.0,0.85,0.85,0.9,1.483333,1.888889,1.240909,0.75,1.375,2.015909,1.55,0.75,0.75,1.15,1.2,1.2,1.340909,0.7,1.919697,1.25,0.75,0.75,0.7,1.858333,0.75,1.875,0.708333,0.75,0.65,1.140909,0.8,0.8,0.75,0.0,0.75,0.8,1.4,1.333333,1.333333,0.65,1.290909,0.75,1.333333,1.966667,0.55,0.8,0.75,0.55,1.175


22
4.0
Max total significant accuracy then averaged 0.5986111111111112 
dataWCV
linear
C: 0.5 


In [32]:


maxAcc = np.max([res for res in np.array(subjectResultArray[subjectResultArray!=None])])
print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
print(np.max([res for res in np.array(subjectResultArray[subjectResultArray!=None])]))

Max single accuracy position (array([1, 1], dtype=int64), array([31, 32], dtype=int64))
0.8999999999999999
